In [1]:
%matplotlib widget

import h5py
import ipywidgets as widgets
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition
from sklearn import manifold
import umap

import plot

matplotlib.__version__

'3.3.2'

In [2]:
SIZE = 2048

with h5py.File("data/data.hdf5", "r") as datafile:
    ids = datafile["id_tr"][:]
    rnd_idx = np.random.randint(ids.shape[0], size=SIZE)
    ids = ids[rnd_idx]
    X = datafile["X_tr"][:][rnd_idx]
    z = datafile["z_tr"][:][rnd_idx].reshape(-1, 1)
ids.shape, X.shape, z.shape

((2048,), (2048, 512), (2048, 1))

In [3]:
fig, ax = plt.subplots()
plot.spectrum(ax, X[0], label="z = " + str(z[0, 0]))
plot.redshift(ax, z[0, 0])
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
output = widgets.Output()

with output:
    fig1, ax1 = plt.subplots(constrained_layout=True)
    fig2, ax2 = plt.subplots(constrained_layout=True)

def project(projection):
    if projection == "PCA":
        # TODO standardization?
        pca = decomposition.PCA(n_components=2)
        embedding = pca.fit_transform(X)
    elif projection == "t-SNE":
        tsne = manifold.TSNE(n_components=2, verbose=2)
        embedding = tsne.fit_transform(X)
    elif projection == "UMAP":
        reducer = umap.UMAP(verbose=True)
        embedding = reducer.fit_transform(X)

    ax1.clear()
    collection = ax1.scatter(embedding[:, 0], embedding[:, 1], c=z, picker=True)
    fig1.colorbar(collection, ax=ax1)
    
widgets.interact_manual(project, projection=widgets.Dropdown(options=["PCA", "t-SNE", "UMAP"]))

def onpick(event):
    ax2.clear()
    plot.spectrum(ax2, X[event.ind[0]])
    ax2.grid(True)

fig1.canvas.mpl_connect("pick_event", onpick)

output

interactive(children=(Dropdown(description='projection', options=('PCA', 't-SNE', 'UMAP'), value='PCA'), Butto…

Output()